In [1]:
import pandas as pd
from data.vesting_data import public_allocations, private_allocations
from utils.daily_charts import *
from utils.api import get_historical_data
from utils.data_processing import convert_vesting_data_to_df

pd.options.mode.chained_assignment = None  # default='warn'

### Data Processing Notebook

In this notebook, we take the simple data in vesting_data.py and turn it into daily unlock tables with corresponding price data, supply data, and other metrics added as columns to aid in our analysis. This creates a large table of data for each token, which is then packaged into an excel notebook with a different sheet for each token.

First, we import the token metadata and convert it into daily tables.

In [2]:
private_allocations = convert_vesting_data_to_df(private_allocations)
public_allocations = convert_vesting_data_to_df(public_allocations)

In [3]:
for name in private_allocations:
    data = private_allocations[name]["vesting"]
    data["start"] = df_column_to_datetime(data["start"])
    data["end"] = df_column_to_datetime(data["end"])
    vest_schedule = create_empty_vest_schedule(data, 'group', 'start', 'end')
    vest_schedule = populate_vest_schedule(data, vest_schedule)
    private_allocations[name]["daily_table"] = vest_schedule

for name in public_allocations:
    data = public_allocations[name]["vesting"]
    data["start"] = df_column_to_datetime(data["start"])
    data["end"] = df_column_to_datetime(data["end"])
    vest_schedule = create_empty_vest_schedule(data, 'group', 'start', 'end')
    vest_schedule = populate_vest_schedule(data, vest_schedule)
    public_allocations[name]["daily_table"] = vest_schedule

For both private allocations and private allocations, we add several new columns, including percent changes in supply and price, circulating supply, total supply, and normalized prices. 

In [4]:

for id in private_allocations:
    table = private_allocations[id]["daily_table"]
    start = table.index[0]
    end = table.index[-1]
    hist_data = get_historical_data(id, start, end)
    
    
    if hist_data.empty:
        continue
    else:
        joined = table.join(hist_data) 
        private_allocations[id]["daily_table"] = joined[joined['price'].notna()] 
        private_allocations[id]["daily_table"] = joined[joined['circulating_supply'].notna()] 
        private_allocations[id]["eth"] = joined[joined['eth'].notna()]
        private_allocations[id]["daily_table"]["p_change_supply"] = (private_allocations[id]["daily_table"]["Daily Vest"] / private_allocations[id]["daily_table"]["circulating_supply"]) * 100
        private_allocations[id]["daily_table"]["p_change_price"] = private_allocations[id]["daily_table"]["price"].pct_change() * 100
        private_allocations[id]["daily_table"]["p_change_macro"] = private_allocations[id]["daily_table"]["eth"].pct_change() * 100
        private_allocations[id]["daily_table"]["normed_price"] = (private_allocations[id]["daily_table"]["price"] - private_allocations[id]["daily_table"]["price"].min()) / (private_allocations[id]["daily_table"]["price"].max() - private_allocations[id]["daily_table"]["price"].min())
        private_allocations[id]["daily_table"]["total_supply"] = private_allocations[id]["total_supply"]

In [5]:

for id in public_allocations:
    table = public_allocations[id]["daily_table"]
    start = table.index[0]
    end = table.index[-1]
    hist_data = get_historical_data(id, start, end)
    
    
    if hist_data.empty:
        continue
    else:
        joined = table.join(hist_data) 
        public_allocations[id]["daily_table"] = joined[joined['price'].notna()] 
        public_allocations[id]["daily_table"] = joined[joined['circulating_supply'].notna()] 
        public_allocations[id]["eth"] = joined[joined['eth'].notna()]
        public_allocations[id]["daily_table"]["p_change_supply"] = (public_allocations[id]["daily_table"]["Daily Vest"] / public_allocations[id]["daily_table"]["circulating_supply"]) * 100
        public_allocations[id]["daily_table"]["p_change_price"] = public_allocations[id]["daily_table"]["price"].pct_change() * 100
        public_allocations[id]["daily_table"]["p_change_macro"] = public_allocations[id]["daily_table"]["eth"].pct_change() * 100
        public_allocations[id]["daily_table"]["normed_price"] = (public_allocations[id]["daily_table"]["price"] - public_allocations[id]["daily_table"]["price"].min()) / (public_allocations[id]["daily_table"]["price"].max() - public_allocations[id]["daily_table"]["price"].min())
        public_allocations[id]["daily_table"]["total_supply"] = public_allocations[id]["total_supply"]

We write both sets to excel notebooks, which are then unpacked and iterated over for analysis in analysis.ipynb.

In [6]:
writer = pd.ExcelWriter("data/daily_unlocks/daily_tables_internal.xlsx", engine="xlsxwriter")

for id in private_allocations:
    private_allocations[id]["daily_table"].to_excel(writer, sheet_name=id)

writer.close()

In [7]:
writer = pd.ExcelWriter("data/daily_unlocks/daily_tables_external.xlsx", engine="xlsxwriter")

for id in public_allocations:
    public_allocations[id]["daily_table"].to_excel(writer, sheet_name=id)

writer.close()